## LLama 3.2 Pipeline 


1) Llama3.2 Vision for OCR on image, outputs text
2) Llama3.2 3b text based LLM for text understanding and formatting into JSON output

In [ ]:
#llama3.2 non vision 3B param test

import base64 
import io
from PIL import Image 
import ollama

from IPython.display import Image as IPImage, Markdown, display
from tqdm import tqdm


import json
from enum import Enum
from pathlib import Path


MODEL = "llama3.2-vision"
TEMPERATURE=0




In [ ]:
class ResponseFormat(Enum):
    JSON = "json_object"
    TEXT = "text"

def call_model(
        prompt: str, response_format: ResponseFormat = ResponseFormat.TEXT
) -> str:
    response = ollama.generate(
        model=MODEL,
        prompt=prompt,
        keep_alive="1h",
        format="" if response_format == ResponseFormat.TEXT else "json",
        options={"temperature": TEMPERATURE}
    )
    return response["response"]

In [78]:
def encode_image_to_base64(image_path : str, format: str = 'PNG') -> str:
    """ Encode image file to base64 string

    Args:
        image_path (str): Path to the image file.
        format (str): Format to save the image in memory (default is PNG).

    Returns:
        str: Base64-encoded image.
    """
    with Image.open(image_path) as img:
        buffered = io.BytesIO()
        img.save(buffered, format=format)
        return base64.b64encode(buffered.getvalue()).decode('utf-8')

def get_ocr_output_from_image(image_base64: str, model: str = "x//llama3.2-vision") -> str:
    """
    Sends image to Llama OCR model and returns text output.

    Args: 
        image_based (str) : Base64-encoded image string.
        mopdel (str) : The model version to use for OCR (default is latest LLama3.2 Vision)
    """
    response = ollama.chat(
        model=model,
        messages=[{
            "role" : "user",
            "content" : "Your task is to extract key information from the image. The receipt will be in jpeg format. Extract the following details: - Store/Merchant name - Date of purchase - Time of purchase - Total amount - Tax amount (if specified) - Payment method - List of items purchased (including quantity and price for each) Provide the extracted information in a JSON format. Follow these guidelines: 1. If any information is unclear or not present in the receipt, use 'N/A' as the value. 2. Format the date as YYYY-MM-DD if possible. 3. Format the time in 24-hour format (HH:MM) if possible. 4. Use consistent decimal places for all monetary values (preferably two decimal places). Please provide the extracted information in JSON format below:",
            "images" : [image_base64]
        }]
    )
    return response.get('message', {}).get('content', '').strip()

In [ ]:
%%time
RECEIPT_PROMPT = f"""Your task is to extract key information from the image. The receipt will be in jpeg format. Extract the following details:
 
- Store/Merchant name
- Date of purchase
- Time of purchase
- Total amount
- Tax amount (if specified)
- Payment method
- List of items purchased (including quantity and price for each)
 
Provide the extracted information in a JSON format. Follow these guidelines:
 
1. If any information is unclear or not present in the receipt, use "N/A" as the value.
2. Format the date as YYYY-MM-DD if possible.
3. Format the time in 24-hour format (HH:MM) if possible.
4. Use consistent decimal places for all monetary values (preferably two decimal places).
 
Please provide the extracted information in JSON format below:"""
 

SyntaxError: f-string: unmatched ')' (<unknown>, line 24)

In [79]:
if __name__ == "__main__":
    image_path = f'C:/Users/Lenovo/Documents/GitHub/Elec376_F24_group7/backend/ml/experiments/1.jpg'
    base64_image = encode_image_to_base64(image_path, format="JPEG")
    ocr_text = get_ocr_output_from_image(base64_image)
    print(ocr_text)

ResponseError: model "x//llama3.2-vision" not found, try pulling it first